In [1]:
import numpy as np
import tifffile
from skimage.filters import threshold_otsu, threshold_local, threshold_niblack, threshold_sauvola
from skimage.morphology import binary_closing, disk
from skimage.exposure import equalize_adapthist, adjust_gamma
import os
from google.colab import drive, files

# First, install SimplITK and pyclesperanto for the specialized operations
!pip install -q SimpleITK
!pip install -q pyclesperanto_prototype

# Import the specialized packages
import SimpleITK as sitk
import pyclesperanto_prototype as cle

# Mount Google Drive
drive.mount('/content/drive')

# Define input and output paths
input_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/denoised/trial'
output_dir = '/content/drive/MyDrive/knowledge/University/Master/Thesis/denoised/output'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Option to upload file directly if not already in Drive
upload_file = False  # Set to True if you want to upload file directly
if upload_file:
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    input_path = filename
else:
    # Use the file from Google Drive
    filename = "denoised_1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq009.tif"
    input_path = os.path.join(input_dir, filename)

print(f"Processing: {filename}")

# Load the image
try:
    img = tifffile.imread(input_path)
    print(f"Image loaded successfully, shape: {img.shape}")
except Exception as e:
    print(f"Error loading image: {e}")
    # If file doesn't exist, upload it
    print("File not found. Please upload the image:")
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    img = tifffile.imread(filename)
    print(f"Uploaded image loaded, shape: {img.shape}")

# Save the original image
tifffile.imwrite(os.path.join(output_dir, "00_original.tif"), img)

# Preprocessing steps
print("Applying preprocessing steps...")

# Step 1: Adaptive histogram equalization (using SimpleITK)
print("1. Applying adaptive histogram equalization...")
try:
    # Using SimpleITK
    sitk_img = sitk.GetImageFromArray(img.astype(np.float32))
    sitk_img_equalized = sitk.AdaptiveHistogramEqualization(sitk_img, alpha=1.0, beta=1.0)
    img_equalized = sitk.GetArrayFromImage(sitk_img_equalized)

    # Alternative using scikit-image if SimpleITK fails
    if np.all(img_equalized == 0):
        raise Exception("SimpleITK equalization failed")

except Exception as e:
    print(f"  SimpleITK approach failed: {e}")
    print("  Using scikit-image equalize_adapthist instead")
    img_equalized = equalize_adapthist(img)

# Save histogram equalized image
tifffile.imwrite(os.path.join(output_dir, "01_adaptive_histogram_equalization.tif"),
                 img_equalized.astype(np.float32))
print("  Saved adaptive histogram equalization result")

# Step 2: Gamma correction
print("2. Applying gamma correction...")
try:
    # Using pyclesperanto if available
    img_gamma = cle.gamma_correction(img_equalized, None, 1.0)
except Exception as e:
    print(f"  Pyclesperanto approach failed: {e}")
    print("  Using scikit-image adjust_gamma instead")
    img_gamma = adjust_gamma(img_equalized, gamma=1.0)

# Save gamma corrected image
tifffile.imwrite(os.path.join(output_dir, "02_gamma_correction.tif"),
                 img_gamma.astype(np.float32))
print("  Saved gamma correction result")

# Step 3: Sum z projection (if image is 3D)
print("3. Checking if sum z projection is needed...")
if len(img_gamma.shape) > 2:
    print(f"  3D image detected with shape {img_gamma.shape}")
    try:
        # Using pyclesperanto if available
        img_projected = cle.sum_z_projection(img_gamma)
    except Exception as e:
        print(f"  Pyclesperanto projection failed: {e}")
        print("  Using numpy sum instead")
        # Determine which axis is the z-axis (usually the first axis)
        z_axis = 0
        img_projected = np.sum(img_gamma, axis=z_axis)

    print(f"  Projected to 2D with shape {img_projected.shape}")
else:
    print("  Image is already 2D, skipping projection")
    img_projected = img_gamma

# Save z-projected image
tifffile.imwrite(os.path.join(output_dir, "03_sum_z_projection.tif"),
                 img_projected.astype(np.float32))
print("  Saved sum z projection result")

# Step 4: Select first channel if needed
print("4. Checking if first channel selection is needed...")
if len(img_projected.shape) > 2:
    if img_projected.shape[2] <= 4:  # Likely RGB/RGBA
        print(f"  Multi-channel image detected with shape {img_projected.shape}")
        # Take first channel
        img_first_channel = img_projected[:, :, 0]
    else:
        # Unusual format, take what's likely the first channel
        img_first_channel = img_projected[:, :, 0]
    print(f"  Selected first channel with shape {img_first_channel.shape}")
else:
    print("  Image is already single channel, no selection needed")
    img_first_channel = img_projected

# Save first channel image
tifffile.imwrite(os.path.join(output_dir, "04_first_channel.tif"),
                 img_first_channel.astype(np.float32))
print("  Saved first channel result")

# Invert the image to detect dark holes
img_max = np.max(img_first_channel)
img_inverted = img_max - img_first_channel
print("5. Image inverted to detect dark holes")

# Save the inverted image
tifffile.imwrite(os.path.join(output_dir, "05_inverted.tif"),
                 img_inverted.astype(np.float32))
print("  Saved inverted image")

# Apply different thresholding techniques for dark spot detection
print("\nApplying thresholding techniques...")

# 1. Otsu thresholding
print("1. Otsu thresholding")
# Apply to original image with < for dark objects
otsu_threshold = threshold_otsu(img_first_channel)
binary_otsu = img_first_channel < otsu_threshold
tifffile.imwrite(os.path.join(output_dir, "06_otsu_threshold.tif"),
                 binary_otsu.astype(np.uint8))
print("  Saved Otsu threshold result")

# 2. Local adaptive thresholding
print("2. Local adaptive thresholding")
block_sizes = [15, 35, 65]
for block_size in block_sizes:
    local_thresh = threshold_local(img_first_channel, block_size, offset=0)
    binary_local = img_first_channel < local_thresh

    # Apply morphological operations to clean up the result
    selem = disk(2)
    binary_local_closed = binary_closing(binary_local, selem)

    tifffile.imwrite(
        os.path.join(output_dir, f"07_local_threshold_block{block_size}.tif"),
        binary_local_closed.astype(np.uint8)
    )
    print(f"  Saved local threshold result (block size {block_size})")

# 3. Niblack thresholding
print("3. Niblack thresholding")
window_size = 25
k_values = [0.1, 0.2, 0.3]
for k in k_values:
    try:
        niblack_thresh = threshold_niblack(img_first_channel, window_size=window_size, k=k)
        binary_niblack = img_first_channel < niblack_thresh

        # Apply morphological operations to clean up the result
        selem = disk(2)
        binary_niblack_closed = binary_closing(binary_niblack, selem)

        tifffile.imwrite(
            os.path.join(output_dir, f"08_niblack_threshold_k{k:.1f}.tif"),
            binary_niblack_closed.astype(np.uint8)
        )
        print(f"  Saved Niblack threshold result (k={k})")
    except Exception as e:
        print(f"  Error with Niblack (k={k}): {e}")

# 4. Sauvola thresholding
print("4. Sauvola thresholding")
window_sizes = [15, 25, 35]
for window_size in window_sizes:
    try:
        sauvola_thresh = threshold_sauvola(img_first_channel, window_size=window_size)
        binary_sauvola = img_first_channel < sauvola_thresh

        # Apply morphological operations to clean up the result
        selem = disk(2)
        binary_sauvola_closed = binary_closing(binary_sauvola, selem)

        tifffile.imwrite(
            os.path.join(output_dir, f"09_sauvola_threshold_w{window_size}.tif"),
            binary_sauvola_closed.astype(np.uint8)
        )
        print(f"  Saved Sauvola threshold result (window size {window_size})")
    except Exception as e:
        print(f"  Error with Sauvola (window_size={window_size}): {e}")

print(f"\nAll results saved to {output_dir}")
print("Processing complete!")

# Show list of output files
print("\nGenerated files:")
for file in sorted(os.listdir(output_dir)):
    print(f" - {file}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.0/624.0 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 724.6/724.6 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.8/92.8 kB 5.9 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/pytools/persistent_dict.py:52: RecommendedHashNotFoundWarning: Unable to import recommended hash 'siphash24.siphash13', falling back to 'hashlib.sha256'. Run 'python3 -m pip install siphash24' to install the recommended hash.
  warn("Unable to import recommended hash 'siphash24.siphash13', "


Mounted at /content/drive
Processing: denoised_1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq009.tif
Error loading image: [Errno 2] No such file or directory: '/content/drive/MyDrive/knowledge/University/Master/Thesis/denoised/trial/denoised_1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq009.tif'
File not found. Please upload the image:


Saving denoised_1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq009.tif to denoised_1.4Pa_A1_20dec21_20xA_L2RA_FlatA_seq009.tif
Uploaded image loaded, shape: (3, 13, 1024, 1024)
Applying preprocessing steps...
1. Applying adaptive histogram equalization...
  SimpleITK approach failed: Exception thrown in SimpleITK AdaptiveHistogramEqualization: /tmp/SimpleITK/Code/Common/include/sitkMemberFunctionFactory.hxx:145:
sitk::ERROR: Pixel type: vector of 32-bit float is not supported in 3D by N3itk6simple40AdaptiveHistogramEqualizationImageFilterE.
  Using scikit-image equalize_adapthist instead


ValueError: Images of type float must be between -1 and 1.